## So sánh

In [84]:
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import torch
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

model = SentenceTransformer("vinai/phobert-base", device='cuda' if torch.cuda.is_available() else 'cpu')

No sentence-transformers model found with name vinai/phobert-base. Creating a new one with mean pooling.


In [64]:
sentence_pairs = [
    ("Tôi thích uống cà phê vào buổi sáng.", "Buổi sáng, tôi hay nhâm nhi ly coffee."),
    ("Trời hôm nay thật đẹp và mát mẻ.", "Thời tiết hôm nay dễ chịu và trong lành."),
    ("Anh ấy đã rời công ty từ tháng trước.", "Tháng rồi, anh ta đã nghỉ việc ở chỗ làm."),
    ("Tôi cảm thấy rất buồn vì kết quả này.", "Kết quả này khiến tôi thất vọng và chán nản."),
    ("Cô ấy là một giáo viên tận tâm và nhiệt huyết.", "Cô ta luôn hết lòng với nghề dạy học.")
]

In [70]:

def get_phobert_embedding(text):
    embedding = model.encode(text)
    return embedding

def cosine_sim_phobert(text1, text2):
    vec1 = get_phobert_embedding(text1).reshape(1, -1)
    vec2 = get_phobert_embedding(text2).reshape(1, -1)
    return cosine_similarity(vec1, vec2)

In [86]:

all_sentences = [s for pair in sentence_pairs for s in pair] 
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

# Thêm CountVectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(all_sentences)

results = []
for idx, (s1, s2) in enumerate(sentence_pairs):
    vec1_tfidf = tfidf_vectorizer.transform([s1])
    vec2_tfidf = tfidf_vectorizer.transform([s2])

    vec1_count = count_vectorizer.transform([s1])
    vec2_count = count_vectorizer.transform([s2])

    cos_tfidf = cosine_similarity(vec1_tfidf, vec2_tfidf)[0][0]
    cos_count = cosine_similarity(vec1_count, vec2_count)[0][0]
    cos_phobert = cosine_sim_phobert(s1, s2)[0][0]

    results.append({
        "Câu 1": s1,
        "Câu 2": s2,
        "CountVectorizer": round(cos_count, 4),
        "TF-IDF": round(cos_tfidf, 4),
        "PhoBERT": round(cos_phobert, 4)
    })

df = pd.DataFrame(results)
df

,Câu 1,Câu 2,CountVectorizer,TF-IDF,PhoBERT
0,Tôi thích uống cà phê vào buổi sáng.,"Buổi sáng, tôi hay nhâm nhi ly coffee.",0.3750,0.2735,0.6888
1,Trời hôm nay thật đẹp và mát mẻ.,Thời tiết hôm nay dễ chịu và trong lành.,0.3536,0.2556,0.6814
2,Anh ấy đã rời công ty từ tháng trước.,"Tháng rồi, anh ta đã nghỉ việc ở chỗ làm.",0.3333,0.2748,0.7316
3,Tôi cảm thấy rất buồn vì kết quả này.,Kết quả này khiến tôi thất vọng và chán nản.,0.4216,0.3331,0.6787
4,Cô ấy là một giáo viên tận tâm và nhiệt huyết.,Cô ta luôn hết lòng với nghề dạy học.,0.1005,0.0791,0.6439


## Demo convert textto token

In [82]:
embedding = model.encode('Nhóm mình đang tìm hiểu phoBERT', normalize_embeddings=True)
embedding.shape # => So chieu bằng (768,)
embedding

array([-7.51986634e-03,  1.70325972e-02, -3.80394422e-02, -1.36641487e-02,
       -3.85071523e-02,  3.87815982e-02, -6.43990040e-02, -1.01122754e-02,
       -3.95898968e-02, -3.98599245e-02,  1.71731878e-02,  1.05369471e-01,
        6.86495507e-04, -1.24018453e-02,  7.97104381e-04,  7.17359362e-03,
        2.98525710e-02,  2.29311883e-02, -1.71285346e-02, -3.32339108e-02,
        4.87042591e-03, -2.75367852e-02, -2.85343118e-02,  5.13379574e-02,
       -3.36058525e-04, -9.00713913e-03,  8.39892924e-02,  2.58513466e-02,
        3.41088474e-02,  4.03110646e-02,  3.46182212e-02,  2.62455344e-02,
        1.55008892e-02, -9.68195591e-03,  4.17049266e-02,  4.64267842e-02,
        1.27256718e-02, -1.14048803e-02,  1.13658356e-02,  8.13027695e-02,
        4.21655774e-02,  8.70336592e-03,  2.13540774e-02,  2.81511806e-02,
        4.22158800e-02,  6.62196502e-02,  1.00167757e-02, -1.77982114e-02,
        1.50276646e-02, -7.36634107e-03, -2.81982310e-02, -1.14704687e-02,
       -4.13219444e-02,  

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
tokenizer.save_pretrained("vinai/phobert-base")
# Xem ID của từng token
print(tokenizer.convert_tokens_to_ids("ganda"))         # Ví dụ: 5678
print(tokenizer.convert_tokens_to_ids("ngôn_viên"))     # Ví dụ: 7890


62465
62466
